# Cell Size Distribution of Microbial Communities

In [ ]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import sys
!conda install --yes --prefix {sys.prefix} sympy
from sympy import *
import scipy as sc
init_printing() #for the equation
from scipy import integrate
import scipy.stats as st
from scipy.stats import beta
import seaborn as sns
import pandas as pd

### Lotka-Volterra model with mass scaling

$\frac{dN}{dt} = N_{i}(Y_{0}V_{i}^{\beta} - \sum \limits A_{0}(V_{i}V_{j})^{\beta}N_{j})$<br>

### Prior distributions

In [ ]:
#prior distributions
plt.style.use(plt.style.available[15]) #select style for plot
plt.rcParams["figure.figsize"] = (10,5.5) #set size of plot

index = np.arange(1,13,1) #place of bars in plot
bars = np.linspace(-3, 3, 13) #create bins (also the name of the ticks)
n = beta.rvs(1,1,loc=-3, scale=6, size=1000000, random_state=None) #log-normally distributed random numbers
r = beta.rvs(1,2,loc=-3, scale=6, size=1000000, random_state=None) #log-right-skewed distributed random numbers
l = beta.rvs(2,1,loc=-3, scale=6, size=1000000, random_state=None) #log-left-skewed distributed random numbers

uniform = np.digitize(n, bars) #sort numbers into bins
uniform_r = np.zeros(13)
for i in range(len(uniform)): #count how often a number fell into each bin
    x = uniform[i]
    uniform_r[x-1] = uniform_r[x-1] + 1
uniform_r = uniform_r/sum(uniform_r) #total = 1

right = np.digitize(r, bars) #sort numbers into bins
right_r = np.zeros(13)
for i in range(len(right)): #count how often a number fell into each bin
    x = right[i]
    right_r[x-1] = right_r[x-1] + 1
right_r = right_r/sum(right_r)

left = np.digitize(l, bars) #sort numbers into bins
left_r = np.zeros(13)
for i in range(len(left)): #count how often a number fell into each bin
    x = left[i]
    left_r[x-1] = left_r[x-1] + 1
left_r = left_r/sum(left_r)



#plot bins 1:12 because bin 13 is empty (no values are larger than 3.0)
sns.kdeplot(n,bw_method=0.1, color='#984ea3', ls='solid', label='log-uniform')
sns.kdeplot(r, bw_method=0.1, color='#a65628', ls='dashed', label='log-right-skewed')
sns.kdeplot(l, bw_method=0.1, color='#999999', ls='dotted', label='log-left-skewed')
plt.legend(loc=9, prop={'size':15})
plt.xlabel('log(cell volume [$\mu m^3$])',fontsize=24)
plt.xlim(-3,3)
plt.ylabel('Density', fontsize=24)
plt.title('Prior Distributions', size=35)
plt.tick_params(axis='both', which='both', labelsize = '15')

plt.savefig('Prior Distribution.png', dpi=300, bbox_inches='tight') #save plot

In [ ]:
#set functions
def sumf(Competition, Species, i): #sums the interactions between species i and all other species
    x = 0
    for j in range(len(Species)):
        x = x + Competition[i][j]*Species[j]
    return(x)

#model and solution
def LV(N1N2, t, Y0, V, b, a): #sets up 1 ODE for each species
    dN_dt = np.empty(Species)
    for i in range(Species):
        dN_dt[i] = N1N2[i]*(Y0*V[i]**b + sumf(a, N1N2, i))
    return dN_dt

#simulation
def simulation(t_vec, Species, Runs, Y0, A0, b, b2, x, y, K): #Simulates the community dynamics 
    bins = np.linspace(-3, 3, 13) #bins for the different size classes
    data = np.zeros(Species) #some empty arrays
    results = np.zeros((Runs+2, len(bins)))
    s=0
    for n in range(Runs): #run simulation n times
        V = 10**beta.rvs(x, y,loc=-3, scale=6,size=Species, random_state=None) #draw volumes from prior distribution
        a = np.zeros((Species,Species)) #empty interaction matrix
        for i in range(Species): #assign the interaction coefficient to each pairwise interaction
            for j in range(Species):
                a[i][j] = -A0*(V[i]*V[j])**b2
        for i in range(Species):
            a[i][i] = K #Diagonals = Carrying Capacity (interaction between individuals of the same species = -100)

        #initial condition
        N10N20 = np.ones(Species)/1000
    
        #ODE Solution
        N1N2_vec = integrate.odeint(LV, N10N20, t_vec, (Y0, V, b, a))
       
        #sort final size distribution into bins and save it in a big array
        for i in range(len(N1N2_vec[10000-1])):
            if N1N2_vec[10000-1][i] < 1*10**(-30):
                N1N2_vec[10000-1][i] = 0
        data = np.log10(V)
        digitized = np.digitize(data, bins)
        for i in range(len(digitized)):
            d = digitized[i]
            results[n][d-1] = results[n][d-1]+N1N2_vec[10000-1][i] 
        
        results[-2] = np.sum(results, axis=0)/Runs #all results added up
        no = sum(results[-2]) #Make all results add up to 1
        results[-2] = results[-2]/no 
        results[-1] = np.std(results[0:Runs]/no, axis=0) #calculate the standard deviation for each size class
        
        #survival
        for i in range(len(N1N2_vec[10000-1])): #calculate how many species have survived until the end
            if N1N2_vec[10000-1][i]>1*10**(-30):
                s=s+1
        survival = 100*((s/Runs)/Species)
        
    print('done')
    return results, survival




### Community Dynamics

In [ ]:
#global parameters
t_vec = np.arange(0, 100., 0.01)
Species = 50 #number of species in the system
Runs=1
Y0 = 5
A0= 0.01
b = 0.73

V = 10**(beta.rvs(1, 1,loc=-3, scale=6,size=Species, random_state=None)) #draw volumes from log-uniform distribution
a = np.zeros((Species, Species)) #empty competition matrix
for i in range(Species): #assign the interaction coefficient to each pairwise interaction
    for j in range(Species):
        a[i][j] = -A0*(V[i]*V[j])**b
for i in range(Species):
    a[i][i] = -100 #Diagonals = -100 (Carrying Capacity)

#initial condition
N10N20 = np.ones(Species)/1000
    
#ODE Solution and Plot
N1N2_vec = integrate.odeint(LV, N10N20, t_vec, (Y0, V, b, a))

#plots
fig, axs = plt.subplots(2,2, figsize=(20,11))
fig.suptitle('Community Dynamics', size=35)
plt.style.use('tableau-colorblind10')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.3, hspace=0.5)
for p in range(len(N1N2_vec)):
    for q in range(Species):
        if N1N2_vec[p][q] < 1*10**-30 or N1N2_vec[p-1][q] == 1*10**-30:
            N1N2_vec[p][q] = 1*10**-30

index = np.arange(1,13,1)
bins = np.linspace(-3, 3, 13) #bins for the different size classes
data = np.zeros(Species) #some empty arrays
results = np.zeros(len(bins))
s=0
data = np.log10(V)
digitized = np.digitize(data, bins)
for i in range(len(digitized)):
    d = digitized[i]
    results[d-1] = results[d-1]+N1N2_vec[10000-1][i] 
no = sum(results) #Make all results add up to 1
results = results/no

#plot A
axs[0,0].plot(t_vec, np.log10(N1N2_vec))
axs[0,0].set_ylabel('log(population density)', fontsize = 24)
axs[0,0].set_xlabel('Time', fontsize=24)
axs[0,0].set_title('A', size=30)
axs[0,0].tick_params(axis='both', which='both', labelsize = '18')
axs[0,0].set_xlim(0,0.5)

#plot B
axs[0,1].hist(np.log10(results[0:12], where=(results[0:12]!=0)), color='#984ea3')
axs[0,1].set_ylabel('Frequency', fontsize=24)
axs[0,1].set_xlabel('log(Population Density)', fontsize=24)
axs[0,1].set_title('B', size=30)
axs[0,1].tick_params(labelsize='18')

#plot C
axs[1,0].plot(t_vec, N1N2_vec)
axs[1,0].set_ylabel('Population density', fontsize = 24)
axs[1,0].set_xlabel('Time', fontsize=24)
axs[1,0].set_title('C', size=30)
axs[1,0].tick_params(axis='both', which='both', labelsize = '18')
axs[1,0].set_xlim(0,0.5)

#plot D
axs[1,1].hist(results[0:12], color='#984ea3')
axs[1,1].set_ylabel('Frequency', fontsize=24)
axs[1,1].set_xlabel('Population Density', fontsize=24)
axs[1,1].set_title('D', size=30)
axs[1,1].tick_params(labelsize='18')
        
#survival
for i in range(len(N1N2_vec[10000-1])): #calculate how many species have survived until the end
    if N1N2_vec[10000-1][i]>1*10**(-30):
        s=s+1
survival = 100*((s/Runs)/Species)   

plt.savefig('4_Dynamics_Plots.png', dpi=300, bbox_inches='tight') #save plot
    
print(round(survival))

In [ ]:
#Community dynamics for right-skewed distribution

#global parameters
t_vec = np.arange(0, 100., 0.01)
Species = 50 #number of species in the system
Runs=1
Y0 = 5
A0= 0.01
b = 0.73

#local Parameters
V = 10**(beta.rvs(1, 2,loc=-3, scale=6,size=Species, random_state=None)) #log-right-skewed prior distribution
a = np.zeros((Species, Species)) #empty competition matrix
for i in range(Species):
    for j in range(Species): #assign the interaction coefficient to each pairwise interaction
        a[i][j] = -A0*(V[i]*V[j])**b
for i in range(Species):
    a[i][i] = -100 #Diagonals = -100 (Carrying Capacity)

#initial condition
N10N20 = np.ones(Species)/1000
    
#ODE Solution and Plot
N1N2_vec = integrate.odeint(LV, N10N20, t_vec, (Y0, V, b, a))

#plots
fig, axs = plt.subplots(2,2, figsize=(20,11))
fig.suptitle('Community Dynamics 2', size=35)
plt.style.use('tableau-colorblind10')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.3, hspace=0.5)
for p in range(len(N1N2_vec)):
    for q in range(Species):
        if N1N2_vec[p][q] < 1*10**-30 or N1N2_vec[p-1][q] == 1*10**-30:
            N1N2_vec[p][q] = 1*10**-30

index = np.arange(1,13,1)
bins = np.linspace(-3, 3, 13) #bins for the different size classes
data = np.zeros(Species) #some empty arrays
results = np.zeros(len(bins))
s=0
data = np.log10(V)
digitized = np.digitize(data, bins)
for i in range(len(digitized)):
    d = digitized[i]
    results[d-1] = results[d-1]+N1N2_vec[10000-1][i] 
no = sum(results) #Make all results add up to 1
results = results/no

#plot A
axs[0,0].plot(t_vec, np.log10(N1N2_vec))
axs[0,0].set_ylabel('log(population density)', fontsize = 24)
axs[0,0].set_xlabel('Time', fontsize=24)
axs[0,0].set_title('A', size=30)
axs[0,0].tick_params(axis='both', which='both', labelsize = '18')
axs[0,0].set_xlim(0,0.5)

#plot B
axs[0,1].hist(np.log10(results[0:12], where=(results[0:12]!=0)), color='#984ea3')
axs[0,1].set_ylabel('Frequency', fontsize=24)
axs[0,1].set_xlabel('log(Population Density)', fontsize=24)
axs[0,1].set_title('B', size=30)
axs[0,1].tick_params(labelsize='18')

#plot C
axs[1,0].plot(t_vec, N1N2_vec)
axs[1,0].set_ylabel('Population density', fontsize = 24)
axs[1,0].set_xlabel('Time', fontsize=24)
axs[1,0].set_title('C', size=30)
axs[1,0].tick_params(axis='both', which='both', labelsize = '18')
axs[1,0].set_xlim(0,0.5)

#plot D
axs[1,1].hist(results[0:12], color='#984ea3')
axs[1,1].set_ylabel('Frequency', fontsize=24)
axs[1,1].set_xlabel('Population Density', fontsize=24)
axs[1,1].set_title('D', size=30)
axs[1,1].tick_params(labelsize='18')
        
#survival
for i in range(len(N1N2_vec[10000-1])): #calculate how many species have survived until the end
    if N1N2_vec[10000-1][i]>1*10**(-30):
        s=s+1
survival = 100*((s/Runs)/Species)   

plt.savefig('4_Dynamics_rightskewed.png', dpi=300, bbox_inches='tight') #save plot
    
print(round(survival))


### Results for different prior distributions

In [ ]:
#global parameters
t_vec = np.arange(0, 100., 0.01)
Species = 50 #number of species in the system
Runs = 50 #number of simulations
Y0 = 5
A0=0.01
b = 0.73
b2 = 0.73

results_n, survival_n = simulation(t_vec, Species, Runs, Y0, A0, b, b2, x=1, y=1, K=-100) #log-uniform prior
results_r, survival_r = simulation(t_vec, Species, Runs, Y0, A0, b, b2, x=1, y=2, K=-100) #log-right-skewed prior
results_l, survival_l = simulation(t_vec, Species, Runs, Y0, A0, b, b2, x=2, y=1, K=-100) #log-left-skewed prior

In [ ]:
print(round(survival_n),round(survival_l),round(survival_r))

index = np.arange(1,13,1)
bars = np.linspace(-3, 3, 13)

fig, axs = plt.subplots(3, sharex='all', figsize=(10,14))
fig.suptitle('Final Cell Size Distributions', size=35)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.4)
#plot A
axs[0].bar(index, results_n[-2][0:12], width = 0.5, align='edge', yerr=results_n[-1][0:12], label='results', 
           color='#984ea3', ecolor='black', capsize=5.0, log=False)
axs[0].plot(index, uniform_r[0:12], label='log-uniform prior', color='#a65628')
axs[0].legend(loc=2, prop={'size':15})
axs[0].set_ylabel('Population density', fontsize=24)
axs[0].set_title('A', size=30)
axs[0].tick_params(labelsize='15')
#plot B
axs[1].bar(index, results_l[-2][0:12], width = 0.5, align='edge', yerr=results_l[-1][0:12], label='results',
           color='#984ea3', ecolor='black', capsize=5.0, log=False)
axs[1].plot(index, left_r[0:12], label='log-left-skewed prior', color='#a65628')
axs[1].legend(loc=2, prop={'size':15})
axs[1].set_ylabel('Population density', fontsize=24)
axs[1].set_title('B', size=30)
axs[1].tick_params(labelsize='15')
#plot C
axs[2].bar(index, results_r[-2][0:12], width = 0.5, align='edge', yerr=results_r[-1][0:12], label='results',
           color='#984ea3', ecolor='black', capsize=5.0, log=False)
axs[2].plot(index, right_r[0:12], label='log-right-skewed prior', color='#a65628')
axs[2].legend(loc=2, prop={'size':15})
axs[2].set_ylabel('Population density', fontsize=24)
axs[2].set_title('C', size=30)
axs[2].tick_params(labelsize='15')

plt.xlabel('log(cell volume [$\mu m^3$])',fontsize=24)
plt.xticks(index, bars[0:12])
axs[2].tick_params(axis='both', which='both', labelsize = '15')

plt.savefig('Results All distributions.png', dpi=300, bbox_inches='tight') #save plot

### Different values for $\beta$

In [ ]:
#global parameters
t_vec = np.arange(0, 100., 0.01)
Species = 50 #number of species in the system
Runs = 50 #number of simulations
Y0 = 1
A0=0.01

#b=0.73 (DeLong)
results_073, survival_073 = simulation(t_vec, Species, Runs, Y0, A0, b=0.73, b2=0.73, x=1, y=1, K=-100)
#b=0.6 (Hira)
results_06, survival_06 = simulation(t_vec, Species, Runs, Y0, A0, b=0.6, b2=0.6, x=1, y=1, K=-100)
#aplha = -0.25 (Kleiber)
results_025, survival_025 = simulation(t_vec, Species, Runs, Y0, A0, b=-0.25, b2=-0.25, x=1, y=1, K=-100)
#b = 1 (very large)
results_1, survival_1 = simulation(t_vec, Species, Runs, Y0, A0, b=1.0, b2=1.0, x=1, y=1, K=-100)
#b = 0.3 (weaker positive scaling)
results_03, survival_03 = simulation(t_vec, Species, Runs, Y0, A0, b=0.3, b2=0.3, x=1, y=1, K=-100)

round(survival_073), round(survival_06), round(survival_025), round(survival_1), round(survival_03)

In [ ]:
#plot
index = np.arange(1,13,1)
bars = np.linspace(-3, 3, 13)

plt.rcParams["figure.figsize"] = (10,5.5)
plt.plot(index, results_1[-2][0:12], label = r'$\beta = 1.0$', color='#a65628', ls='dashed')
plt.plot(index, results_073[-2][0:12], label=r'$\beta = 0.73$', color='#984ea3', ls='solid')
plt.plot(index, results_06[-2][0:12], label=r'$\beta = 0.6$', color='#999999', ls='dotted')
plt.plot(index, results_03[-2][0:12], label=r'$\beta = 0.3$', color='#e41a1c', ls='dashdot')
plt.plot(index, results_025[-2][0:12], label=r'$\beta = -0.25$', color='#dede00', ls=(0,(3,5,1,5,1,5)))
plt.xticks(index, bars[0:12])
plt.legend(loc=2, prop={'size':15})
plt.xlabel('log(cell volume [$\mu m^3$])',fontsize=24)
plt.ylabel('Population density', fontsize=24)
plt.title(r'A: Results for 5 Values of $\beta$', size=35)
plt.tick_params(axis='both', which='both', labelsize = '15')

plt.savefig('different beta.png', dpi=300, bbox_inches='tight') #save plot

### $\beta_{2}$ is different from $\beta$

In [ ]:
#global parameters
t_vec = np.arange(0, 100., 0.01)
Species = 50 #number of species in the system
Runs = 50 #number of simulations
Y0 = 5
A0=0.01
b = 0.73

#b2=0.73
results_73, survival_73 = simulation(t_vec, Species, Runs, Y0, A0, b, b2=0.73, x=1, y=1, K=-100)
#b2=0.5
results_5, survival_5 = simulation(t_vec, Species, Runs, Y0, A0, b, b2=0.5, x=1, y=1, K=-100)
#b2=0.2
results_2, survival_2 = simulation(t_vec, Species, Runs, Y0, A0, b, b2=0.2, x=1, y=1, K=-100)
#b2=-0.25
results_025, survival_025 = simulation(t_vec, Species, Runs, Y0, A0, b, b2=-0.25, x=1, y=1, K=-100)
#b2=-0.5
results_05, survival_05 = simulation(t_vec, Species, Runs, Y0, A0, b, b2=-0.5, x=1, y=1, K=-100)

round(survival_73), round(survival_5), round(survival_2), round(survival_025), round(survival_05)

In [ ]:
#plot
index = np.arange(1,13,1)
bars = np.linspace(-3, 3, 13)

plt.rcParams["figure.figsize"] = (10,5.5)
plt.plot(index, results_73[-2][0:12], label=r'$\beta _{2} = 0.73$', color='#984ea3', ls='solid')
plt.plot(index, results_5[-2][0:12], label=r'$\beta _{2} = 0.5$', color='#a65628', ls='dotted')
plt.plot(index, results_2[-2][0:12], label=r'$\beta _{2} = 0.2$', color='#999999', ls='dashed')
plt.plot(index, results_025[-2][0:12], label=r'$\beta _{2} = -0.25$', color='#e41a1c', ls='dashdot')
plt.plot(index, results_05[-2][0:12], label=r'$\beta _{2} = -0.5$', color='#dede00', ls=(0,(3,5,1,5,1,5)))
plt.xticks(index, bars[0:12])
plt.legend(loc=2, prop={'size':15})
plt.xlabel('log(cell volume [$\mu m^3$])',fontsize=24)
plt.ylabel('Population density', fontsize=24)
plt.title(r'B: Results for 5 Values of $\beta _{2}$', size=35)
plt.tick_params(axis='both', which='both', labelsize = '15')

plt.savefig('different beta2', dpi=300, bbox_inches='tight') #save plot

### Different values for $Y_{0}$ (rough) 

In [ ]:
#global parameters
t_vec = np.arange(0, 100., 0.01)
Species = 50 #number of species in the system
Runs = 50 #number of simulations
Y0 = 5

#Y0 = 10
results_10, survival_10 = simulation(t_vec, Species, Runs, Y0=10, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)
#Y0 = 1
results_1, survival_1 = simulation(t_vec, Species, Runs, Y0=1, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)
#Y0 = 0.1
results_01, survival_01 = simulation(t_vec, Species, Runs, Y0=0.1, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)
#Y0 = 0.01
results_001, survival_001 = simulation(t_vec, Species, Runs, Y0=0.01, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)
#Y0 = 0.001
results_0001, survival_0001 = simulation(t_vec, Species, Runs, Y0=0.001, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)

round(survival_10), round(survival_1), round(survival_01), round(survival_001), round(survival_0001)

In [ ]:
#plot
index = np.arange(1,13,1)
bars = np.linspace(-3, 3, 13)

plt.rcParams["figure.figsize"] = (10,5.5)
plt.plot(index, results_10[-2][0:12], label = r'$Y_{0} = 10$', color='#a65628', ls='solid')
plt.plot(index, results_1[-2][0:12], label=r'$Y_{0} = 1$', color='#984ea3', ls='dashed')
plt.plot(index, results_01[-2][0:12], label=r'$Y_{0} = 0.1$', color='#999999', ls='dotted')
plt.plot(index, results_001[-2][0:12], label=r'$Y_{0} = 0.01$', color='#e41a1c', ls=(0,(3,5,1,5,1,5)))
plt.plot(index, results_0001[-2][0:12], label=r'$Y_{0} = 0.001$', color='#dede00', ls='dashdot')
plt.xticks(index, bars[0:12])
plt.legend(loc=2, prop={'size':15})
plt.xlabel('log(cell volume [$\mu m^3$])',fontsize=24)
plt.ylabel('Population density', fontsize=24)
plt.title(r'Results for 5 Values of $Y_{0}$ (rough)', size=35)
plt.tick_params(axis='both', which='both', labelsize = '15')

plt.savefig('different Y0 rough.png', dpi=300, bbox_inches='tight') #save plot

### Different Values for $Y_{0}$ (fine-tuned)

In [ ]:
#global parameters
t_vec = np.arange(0, 100., 0.01)
Species = 50 #number of species in the system
Runs = 50 #number of simulations
Y0 = 5
A0= 0.01
b = 0.73

#Y0 = 10
results_10, survival_10 = simulation(t_vec, Species, Runs, Y0=10, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)
#Y0 = 7.5
results_75, survival_75 = simulation(t_vec, Species, Runs, Y0=7.5, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)
#Y0 = 5
results_5, survival_5 = simulation(t_vec, Species, Runs, Y0=5, A0=0.01, b=0.73, x=1, b2=0.73, y=1, K=-100)
#Y0 = 2.5
results_25, survival_25 = simulation(t_vec, Species, Runs, Y0=2.5, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)
#Y0 = 1
results_1, survival_1 = simulation(t_vec, Species, Runs, Y0=1, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)

round(survival_10), round(survival_75), round(survival_5), round(survival_25), round(survival_1)

In [ ]:
#plot
index = np.arange(1,13,1)
bars = np.linspace(-3, 3, 13)

plt.rcParams["figure.figsize"] = (10,5.5)
plt.plot(index, results_10[-2][0:12], label = r'$Y_{0} = 10$', color='#a65628', ls='solid')
plt.plot(index, results_75[-2][0:12], label=r'$Y_{0} = 7.5$', color='#984ea3', ls='dashed')
plt.plot(index, results_5[-2][0:12], label=r'$Y_{0} = 5$', color='#999999', ls='dotted')
plt.plot(index, results_25[-2][0:12], label=r'$Y_{0} = 2.5$', color='#e41a1c', ls='dashdot')
plt.plot(index, results_1[-2][0:12], label=r'$Y_{0} = 1$', color='#dede00', ls=(0,(3,5,1,5,1,5)))
plt.xticks(index, bars[0:12])
plt.legend(loc=2, prop={'size':15})
plt.xlabel('log(cell volume [$\mu m^3$])',fontsize=24)
plt.ylabel('Population density', fontsize=24)
plt.title(r'Results for 5 Values of $Y_{0}$ (fine)', size=35)
plt.tick_params(axis='both', which='both', labelsize = '15')

plt.savefig('different Y0 fine', dpi=300, bbox_inches='tight') #save plot

### Different values for $A_{0}$

In [ ]:
#global parameters
t_vec = np.arange(0, 100., 0.01)
Species = 50 #number of species in the system
Runs = 50 #number of simulations
Y0 = 5
A0= 0.01
b = 0.73
Y0 = 5

#A0 = 100
results_100, survival_100 = simulation(t_vec, Species, Runs, Y0, A0=100, b=0.73, b2=0.73, x=1, y=1, K=-100)
#A0 = 10
results_10, survival_10 = simulation(t_vec, Species, Runs, Y0, A0=10, b=0.73, b2=0.73, x=1, y=1, K=-100)
#A0 = 1
results_1, survival_1 = simulation(t_vec, Species, Runs, Y0, A0=1, b=0.73, b2=0.73, x=1, y=1, K=-100)
#A0 = 0.1
results_01, survival_01 = simulation(t_vec, Species, Runs, Y0, A0=0.1, b=0.73, b2=0.73, x=1, y=1, K=-100)
#A0 = 0.01
results_001, survival_001 = simulation(t_vec, Species, Runs, Y0, A0=0.01, b=0.73, b2=0.73, x=1, y=1, K=-100)
#A0 = 0.001
results_0001, survival_0001 = simulation(t_vec, Species, Runs, Y0, A0=0.001, b=0.73, b2=0.73, x=1, y=1, K=-100)

round(survival_100), round(survival_10),round(survival_1), round(survival_01), round(survival_001), round(survival_0001)

In [ ]:
#plot
index = np.arange(1,13,1)
bars = np.linspace(-3, 3, 13)

plt.rcParams["figure.figsize"] = (10,5.5)
plt.plot(index, results_100[-2][0:12], label=r'$A_{0} = 100$', color='#f781bf', ls=(0,(5,1)))
plt.plot(index, results_10[-2][0:12], label=r'$A_{0} = 10$', color='#dede00', ls=(0,(3,5,1,5,1,5)))
plt.plot(index, results_1[-2][0:12], label=r'$A_{0} = 1$', color='#e41a1c', ls='dashdot')
plt.plot(index, results_01[-2][0:12], label=r'$A_{0} = 0.1$', color='#999999', ls='dotted')
plt.plot(index, results_001[-2][0:12], label=r'$A_{0} = 0.01$', color='#984ea3', ls='solid')
plt.plot(index, results_0001[-2][0:12], label=r'$A_{0} = 0.001$', color='#a65628', ls='dashed')
plt.xticks(index, bars[0:12])
plt.legend(loc=2,prop={'size':15})
plt.xlabel('log(cell volume [$\mu m^3$])',fontsize=24)
plt.ylabel('Population density', fontsize=24)
plt.title(r'Results for 6 Values of $A_{0}$', size=35)
plt.tick_params(axis='both', which='both', labelsize = '15')

plt.savefig('different A0', dpi=300, bbox_inches='tight') #save plot

### Different Values for $a_{ii}$ (Carrying capacity)

In [ ]:
#global parameters
t_vec = np.arange(0, 100., 0.01)
Species = 50 #number of species in the system
Runs = 50 #number of simulations
Y0 = 5
A0=0.01
b = 0.73
b2 = 0.73
x=1
y=1

#K=-100
results_100, survival_100 = simulation(t_vec, Species, Runs, Y0, A0, b, b2, x, y, K=-100)
#K=-10
results_10, survival_10 = simulation(t_vec, Species, Runs, Y0, A0, b, b2, x, y, K=-10)
#K=-1
results_1, survival_1 = simulation(t_vec, Species, Runs, Y0, A0, b, b2, x, y, K=-1)
#K=-0.1
results_01, survival_01 = simulation(t_vec, Species, Runs, Y0, A0, b, b2, x, y, K=-0.1)
#K=-0.01
results_001, survival_001 = simulation(t_vec, Species, Runs, Y0, A0, b, b2, x, y, K=-0.01)

round(survival_100), round(survival_10), round(survival_1), round(survival_01), round(survival_001)

In [ ]:
#plot
index = np.arange(1,13,1)
bars = np.linspace(-3, 3, 13)

plt.rcParams["figure.figsize"]=(10,5.5)
plt.plot(index, results_73[-2][0:12], label=r'$a_{ii} = -100$', color='#984ea3', ls='solid')
plt.plot(index, results_5[-2][0:12], label=r'$a_{ii}  = -10$', color='#a65628', ls='dotted')
plt.plot(index, results_2[-2][0:12], label=r'$a_{ii}  = -1$', color='#999999', ls='dashed')
plt.plot(index, results_025[-2][0:12], label=r'$a_{ii} = -0.1$', color='#e41a1c', ls='dashdot')
plt.plot(index, results_05[-2][0:12], label=r'$a_{ii} = -0.01$', color='#dede00', ls=(0,(3,5,1,5,1,5)))
plt.xticks(index, bars[0:12])
plt.legend()
plt.xlabel('log(cell volume [$\mu m^3$])',fontsize=24)
plt.ylabel('Population density', fontsize=24)
plt.title(r'Results for 5 Values of $a_{ii}$', size=35)
plt.tick_params(axis='both', which='both', labelsize = '15')

plt.savefig('different aii', dpi=300, bbox_inches='tight') #save plot